<a href="https://colab.research.google.com/github/antussa2016/thesis_work/blob/master/model/thesis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/antussa2016/thesis_work

Cloning into 'thesis_work'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 107 (delta 41), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (107/107), 377.70 KiB | 2.15 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [81]:
df = pd.read_csv('thesis_work/dataset/data_for_model.csv')

In [82]:
df.head(1)

,twitter_id,stackoverflow_id,question_count,answer_count,reputation,answer_tag,question_tag,Openness,Conscientiousness,Extraversion,...,preferred_tags1,preferred_tags2,preferred_tags3,micro-oop,oop,web-pro,database,mobile,statistic,other
0,10F1,145587,13,500,74297,"['go', 'json', 'http', 'goroutine', 'struct', ...","['go', 'javascript', 'python', 'c++', 'linux',...",0.839299,0.170988,0.083984,...,go,javascript,json,0,0,1,0,0,1,0


In [83]:
df.drop(columns=['twitter_id','stackoverflow_id','question_count','answer_count','answer_tag','question_tag'], inplace=True)

In [84]:
df.head(1)

,reputation,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,preferred_tags1,preferred_tags2,preferred_tags3,micro-oop,oop,web-pro,database,mobile,statistic,other
0,74297,0.839299,0.170988,0.083984,0.047367,0.612101,go,javascript,json,0,0,1,0,0,1,0


In [85]:
# Calculate min and max values for the column
min_value = df['reputation'].min()
max_value = df['reputation'].max()

# Apply min-max scaling to the column
df['reputation'] = (df[['reputation']] - min_value) / (max_value - min_value)

df.head(1)

,reputation,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,preferred_tags1,preferred_tags2,preferred_tags3,micro-oop,oop,web-pro,database,mobile,statistic,other
0,0.057832,0.839299,0.170988,0.083984,0.047367,0.612101,go,javascript,json,0,0,1,0,0,1,0


In [86]:
# Drop rows where any row contains 0 in 'Column1'
df = df[df['preferred_tags1'] != '0']

# Print the updated Dat
df

,reputation,Openness,Conscientiousness,Extraversion,Agreeableness,Neuroticism,preferred_tags1,preferred_tags2,preferred_tags3,micro-oop,oop,web-pro,database,mobile,statistic,other
0,0.057832,0.839299,0.170988,0.083984,0.047367,0.612101,go,javascript,json,0,0,1,0,0,1,0
1,0.036133,0.849555,0.100009,0.032557,0.001654,0.392036,c#,.net,f#,1,0,0,0,0,0,0
2,0.079206,0.894954,0.124989,0.083294,0.009011,0.288220,c++,python,javascript,0,1,1,0,0,0,0
3,0.078168,0.689216,0.625448,0.197740,0.037679,0.223963,c#,java,.net,1,1,0,0,0,0,0
4,0.086798,0.881779,0.042616,0.072976,0.003885,0.432516,c#,ruby,.net,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,0.003969,0.888007,0.739341,0.030510,0.037764,0.342692,angularjs,typescript,javascript,0,0,1,0,0,0,0
815,0.239725,0.596079,0.105716,0.301169,0.245279,1.000000,.net,python,jquery,1,1,1,0,0,0,0
816,0.041576,0.969066,0.055010,0.155452,0.028997,0.295512,javascript,python,xml,1,1,1,0,0,0,0
817,0.051964,0.917028,0.062395,0.017340,0.000687,0.461590,python,django,css,0,1,1,0,0,0,0


In [87]:
df = df[df['preferred_tags2'] != '0']

In [88]:
df = df[df['preferred_tags3'] != '0']

In [ ]:
df.isnull().sum()

In [89]:
df.preferred_tags1=pd.Categorical(df.preferred_tags1,['.net', 'c#', 'f#','asp.net','vb.net','xml','java', 'python', 'swing','c++','c','php', 'ruby', 'perl','jquery','javascript','node.js','angularjs','reactjs','typescript','html','css','json','django','sql', 'sql-server', 'mysql','kotlin', 'objective-c', 'swift','android','ios','go', 'scala', 'r','git', 'github', 'os','host'],ordered=False)


In [90]:
df.preferred_tags1=df.preferred_tags1.cat.codes

In [91]:
df.preferred_tags2=pd.Categorical(df.preferred_tags2,['.net', 'c#', 'f#','asp.net','vb.net','xml','java', 'python', 'swing','c++','c','php', 'ruby', 'perl','jquery','javascript','node.js','angularjs','reactjs','typescript','html','css','json','django','sql', 'sql-server', 'mysql','kotlin', 'objective-c', 'swift','android','ios','go', 'scala', 'r','git', 'github', 'os','host'],ordered=False)

In [92]:
df.preferred_tags2=df.preferred_tags2.cat.codes

In [93]:
df.preferred_tags3=pd.Categorical(df.preferred_tags3,['.net', 'c#', 'f#','asp.net','vb.net','xml','java', 'python', 'swing','c++','c','php', 'ruby', 'perl','jquery','javascript','node.js','angularjs','reactjs','typescript','html','css','json','django','sql', 'sql-server', 'mysql','kotlin', 'objective-c', 'swift','android','ios','go', 'scala', 'r','git', 'github', 'os','host'],ordered=False)

In [94]:
df.preferred_tags3=df.preferred_tags3.cat.codes

In [ ]:
df.preferred_tags1.value_counts()

In [52]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to the 'Programming Tags' column
df['preferred_tags1'] = label_encoder.fit_transform(df['preferred_tags1'])

In [53]:
class_labels = label_encoder.classes_

In [54]:
label_mapping = {class_label: encoded_value for class_label, encoded_value in zip(class_labels, range(len(class_labels)))}

# Print the class labels and their corresponding encoded values
for class_label, encoded_value in label_mapping.items():
    print(f'Class Label: {class_label}, Encoded Value: {encoded_value}')

Class Label: .net, Encoded Value: 0
Class Label: android, Encoded Value: 1
Class Label: angularjs, Encoded Value: 2
Class Label: asp.net, Encoded Value: 3
Class Label: c, Encoded Value: 4
Class Label: c#, Encoded Value: 5
Class Label: c++, Encoded Value: 6
Class Label: css, Encoded Value: 7
Class Label: django, Encoded Value: 8
Class Label: f#, Encoded Value: 9
Class Label: git, Encoded Value: 10
Class Label: go, Encoded Value: 11
Class Label: host, Encoded Value: 12
Class Label: html, Encoded Value: 13
Class Label: ios, Encoded Value: 14
Class Label: java, Encoded Value: 15
Class Label: javascript, Encoded Value: 16
Class Label: jquery, Encoded Value: 17
Class Label: json, Encoded Value: 18
Class Label: kotlin, Encoded Value: 19
Class Label: mysql, Encoded Value: 20
Class Label: node.js, Encoded Value: 21
Class Label: objective-c, Encoded Value: 22
Class Label: os, Encoded Value: 23
Class Label: perl, Encoded Value: 24
Class Label: php, Encoded Value: 25
Class Label: python, Encoded V

In [55]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to the 'Programming Tags' column
df['preferred_tags2'] = label_encoder.fit_transform(df['preferred_tags2'])

In [56]:
class_labels = label_encoder.classes_
label_mapping = {class_label: encoded_value for class_label, encoded_value in zip(class_labels, range(len(class_labels)))}

# Print the class labels and their corresponding encoded values
for class_label, encoded_value in label_mapping.items():
    print(f'Class Label: {class_label}, Encoded Value: {encoded_value}')

Class Label: .net, Encoded Value: 0
Class Label: android, Encoded Value: 1
Class Label: angularjs, Encoded Value: 2
Class Label: asp.net, Encoded Value: 3
Class Label: c, Encoded Value: 4
Class Label: c#, Encoded Value: 5
Class Label: c++, Encoded Value: 6
Class Label: css, Encoded Value: 7
Class Label: django, Encoded Value: 8
Class Label: git, Encoded Value: 9
Class Label: host, Encoded Value: 10
Class Label: html, Encoded Value: 11
Class Label: ios, Encoded Value: 12
Class Label: java, Encoded Value: 13
Class Label: javascript, Encoded Value: 14
Class Label: jquery, Encoded Value: 15
Class Label: json, Encoded Value: 16
Class Label: kotlin, Encoded Value: 17
Class Label: mysql, Encoded Value: 18
Class Label: node.js, Encoded Value: 19
Class Label: objective-c, Encoded Value: 20
Class Label: os, Encoded Value: 21
Class Label: perl, Encoded Value: 22
Class Label: php, Encoded Value: 23
Class Label: python, Encoded Value: 24
Class Label: r, Encoded Value: 25
Class Label: reactjs, Encod

In [57]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to the 'Programming Tags' column
df['preferred_tags3'] = label_encoder.fit_transform(df['preferred_tags3'])

In [58]:
class_labels = label_encoder.classes_
label_mapping = {class_label: encoded_value for class_label, encoded_value in zip(class_labels, range(len(class_labels)))}

# Print the class labels and their corresponding encoded values
for class_label, encoded_value in label_mapping.items():
    print(f'Class Label: {class_label}, Encoded Value: {encoded_value}')

Class Label: .net, Encoded Value: 0
Class Label: android, Encoded Value: 1
Class Label: angularjs, Encoded Value: 2
Class Label: asp.net, Encoded Value: 3
Class Label: c, Encoded Value: 4
Class Label: c#, Encoded Value: 5
Class Label: c++, Encoded Value: 6
Class Label: css, Encoded Value: 7
Class Label: django, Encoded Value: 8
Class Label: f#, Encoded Value: 9
Class Label: git, Encoded Value: 10
Class Label: github, Encoded Value: 11
Class Label: go, Encoded Value: 12
Class Label: host, Encoded Value: 13
Class Label: html, Encoded Value: 14
Class Label: ios, Encoded Value: 15
Class Label: java, Encoded Value: 16
Class Label: javascript, Encoded Value: 17
Class Label: jquery, Encoded Value: 18
Class Label: json, Encoded Value: 19
Class Label: kotlin, Encoded Value: 20
Class Label: mysql, Encoded Value: 21
Class Label: node.js, Encoded Value: 22
Class Label: objective-c, Encoded Value: 23
Class Label: os, Encoded Value: 24
Class Label: perl, Encoded Value: 25
Class Label: php, Encoded V